In [1]:
import pandas as pd
import itertools
import pulp


In [2]:
import numpy as np

In [3]:
df = pd.read_csv('filtered3_df_full.csv')
pred = pd.read_csv('filtered3_prediction_cutoff_1_16.csv')
pred = pred[['PLAYER_NAME', 'PRED_SCORE']]
df.columns.values
df = df[['Season','Game_ID', 'PLAYER_NAME', 'POS', 'Team', 'GAME_DATE', 'MATCHUP',
       'WL', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT',
       'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK',
       'TOV', 'PF', 'PTS', 'PLUS_MINUS', 'VIDEO_AVAILABLE', 'DD', 'TD',
       'total_fantasy_points', 'salary']]


In [4]:
# DO NOT RUN 
def get_recent_salary(df):
    # sort by game date in descending order
    df_sorted = df.sort_values(by=['GAME_DATE'], ascending=False)
    
    # group by player name and get first row of each group to get player's most recent game 
    df_recent_salary = df_sorted.groupby('PLAYER_NAME').first()
    
    return df_recent_salary[['salary']].reset_index()

df_recent_salary = get_recent_salary(df)




In [4]:
# Example lineup for game date 2021-1-16
# RUN
df_ex = df[(df['GAME_DATE'] == '2021-01-16')]
merged_df = pd.merge(pred, df_ex[['PLAYER_NAME', 'salary', 'POS', 'total_fantasy_points']], on='PLAYER_NAME')
merged_df
merged_df.columns.values
merged_df = merged_df.drop_duplicates(subset=['PLAYER_NAME'], keep='first')
merged_df


,PLAYER_NAME,PRED_SCORE,salary,POS,total_fantasy_points
0,Aaron Gordon,26.337820,7400.0,PF,27.50
1,Bam Adebayo,36.686985,8600.0,C,35.25
2,Bismack Biyombo,23.806379,4000.0,C,27.25
3,Chris Silva,15.596848,3400.0,C,7.25
4,Christian Wood,25.886425,8700.0,C,46.00
5,Clint Capela,31.027426,6700.0,C,43.25
6,Damian Lillard,27.056828,9200.0,PG,36.25
7,DeAndre Jordan,27.208178,4500.0,C,18.75
8,Dejounte Murray,32.388432,7800.0,PG,35.50
9,Derrick Rose,13.274501,5200.0,PG,28.75


In [5]:
filtered_df = merged_df[merged_df['POS'] == 'PG']
min_value = filtered_df['salary'].min()
print(f"Minimum salary for PG: {min_value}")
least_salary_rows = filtered_df.nsmallest(5, 'salary')
print(least_salary_rows)
filtered_df = merged_df[merged_df['POS'] == 'SG']
min_value = filtered_df['salary'].min()
print(f"Minimum salary for SG: {min_value}")
least_salary_rows = filtered_df.nsmallest(5, 'salary')
print(least_salary_rows)
filtered_df = merged_df[merged_df['POS'] == 'SF']
min_value = filtered_df['salary'].min()
print(f"Minimum salary for SF: {min_value}")
least_salary_rows = filtered_df.nsmallest(5, 'salary')
print(least_salary_rows)
filtered_df = merged_df[merged_df['POS'] == 'PF']
min_value = filtered_df['salary'].min()
print(f"Minimum salary for PF: {min_value}")
least_salary_rows = filtered_df.nsmallest(5, 'salary')
print(least_salary_rows)
filtered_df = merged_df[merged_df['POS'] == 'C']
min_value = filtered_df['salary'].min()
print(f"Minimum salary for C: {min_value}")
least_salary_rows = filtered_df.nsmallest(5, 'salary')
print(least_salary_rows)


Minimum salary for PG: 3000.0
      PLAYER_NAME  PRED_SCORE  salary POS  total_fantasy_points
44      Saben Lee   11.982996  3000.0  PG                  4.50
28  Malachi Flynn    9.171875  3300.0  PG                 14.25
9    Derrick Rose   13.274501  5200.0  PG                 28.75
39     Tyus Jones   17.954525  5800.0  PG                 19.75
41   Cole Anthony    3.852463  5900.0  PG                 34.75
Minimum salary for SG: 3000.0
        PLAYER_NAME  PRED_SCORE  salary POS  total_fantasy_points
42       Malik Monk    5.365750  3000.0  SG                  4.00
27    Landry Shamet   18.961679  3300.0  SG                 10.25
35        Max Strus   20.301575  3300.0  SG                  6.00
14       Isaiah Joe   18.789894  4100.0  SG                  8.50
13  Duncan Robinson   16.337984  5400.0  SG                 24.50
Minimum salary for SF: 3000.0
         PLAYER_NAME  PRED_SCORE  salary POS  total_fantasy_points
32  Matisse Thybulle   17.201925  3000.0  SF                 22

In [6]:
# WITHOUT SALARY CONSTRAINT using pred score

def optimize_lineup(merged_df):
    # filter by positions
    PG = merged_df[merged_df['POS'] == 'PG']
    SG = merged_df[merged_df['POS'] == 'SG']
    SF = merged_df[merged_df['POS'] == 'SF']
    PF = merged_df[merged_df['POS'] == 'PF']
    C = merged_df[merged_df['POS'] == 'C']

    # define positions and # of players for each position
    positions = {'PG': 1, 'SG': 1, 'SF': 1, 'PF': 1, 'C': 1, 'G': 1, 'F': 1, 'Util': 1}

    # initialize the lineup dictionary
    lineup = {position: [] for position in positions.keys()}

    # loop through the sorted dataframes for each position
    for pos, df in {'PG': PG, 'SG': SG, 'SF': SF, 'PF': PF, 'C': C}.items():
        df = df.sort_values(by='PRED_SCORE', ascending=False)

        for idx, player in df.iterrows():
            if positions[pos] > 0 and sum(positions.values()) > 1:
                lineup[pos].append(player['PLAYER_NAME'])
                positions[pos] -= 1

    # fill the G, F, and Util spots
    all_positions_df = merged_df.sort_values(by='PRED_SCORE', ascending=False)
    for idx, player in all_positions_df.iterrows():
        pos = player['POS']

        if positions['G'] > 0 and pos in {'PG', 'SG'} and player['PLAYER_NAME'] not in lineup['PG'] + lineup['SG']:
            lineup['G'].append(player['PLAYER_NAME'])
            positions['G'] -= 1
        elif positions['F'] > 0 and pos in {'SF', 'PF'} and player['PLAYER_NAME'] not in lineup['SF'] + lineup['PF']:
            lineup['F'].append(player['PLAYER_NAME'])
            positions['F'] -= 1
        elif positions['Util'] > 0 and player['PLAYER_NAME'] not in (lineup['PG'] + lineup['SG'] + lineup['SF'] +
                                                                     lineup['PF'] + lineup['C'] + lineup['G'] +
                                                                     lineup['F']):
            lineup['Util'].append(player['PLAYER_NAME'])
            positions['Util'] -= 1

        if all(count == 0 for count in positions.values()):
            break

    total_score = sum(merged_df.loc[merged_df['PLAYER_NAME'].isin(lineup[pos]), 'PRED_SCORE'].sum() for pos in lineup.keys())
    total_salary = sum(merged_df.loc[merged_df['PLAYER_NAME'].isin(lineup[pos]), 'salary'].sum() for pos in lineup.keys())

    # print the lineup and the total score and total salary
    print('Lineup:')
    for position, players in lineup.items():
        print(f'{position}: {", ".join(players)}')
    print(f'Total score: {total_score}')
    print(f'Total salary: {total_salary}')

optimize_lineup(merged_df)


Lineup:
PG: Kyle Lowry
SG: Max Strus
SF: Yuta Watanabe
PF: Kevin Durant
C: Bam Adebayo
G: Dejounte Murray
F: Aaron Gordon
Util: Devonte' Graham
Total score: 230.62527656555176
Total salary: 53900.0


In [7]:
# WITHOUT SALARY CONSTRAINT using total fantasy points

def optimize_lineup(merged_df):
    # filter by positions
    PG = merged_df[merged_df['POS'] == 'PG']
    SG = merged_df[merged_df['POS'] == 'SG']
    SF = merged_df[merged_df['POS'] == 'SF']
    PF = merged_df[merged_df['POS'] == 'PF']
    C = merged_df[merged_df['POS'] == 'C']

    # define positions and # of players for each position
    positions = {'PG': 1, 'SG': 1, 'SF': 1, 'PF': 1, 'C': 1, 'G': 1, 'F': 1, 'Util': 1}

    # initialize the lineup dictionary
    lineup = {position: [] for position in positions.keys()}

    # loop through the sorted dataframes for each position
    for pos, df in {'PG': PG, 'SG': SG, 'SF': SF, 'PF': PF, 'C': C}.items():
        df = df.sort_values(by='total_fantasy_points', ascending=False)

        for idx, player in df.iterrows():
            if positions[pos] > 0 and sum(positions.values()) > 1:
                lineup[pos].append(player['PLAYER_NAME'])
                positions[pos] -= 1

    # fill the G, F, and Util spots
    all_positions_df = merged_df.sort_values(by='total_fantasy_points', ascending=False)
    for idx, player in all_positions_df.iterrows():
        pos = player['POS']

        if positions['G'] > 0 and pos in {'PG', 'SG'} and player['PLAYER_NAME'] not in lineup['PG'] + lineup['SG']:
            lineup['G'].append(player['PLAYER_NAME'])
            positions['G'] -= 1
        elif positions['F'] > 0 and pos in {'SF', 'PF'} and player['PLAYER_NAME'] not in lineup['SF'] + lineup['PF']:
            lineup['F'].append(player['PLAYER_NAME'])
            positions['F'] -= 1
        elif positions['Util'] > 0 and player['PLAYER_NAME'] not in (lineup['PG'] + lineup['SG'] + lineup['SF'] +
                                                                     lineup['PF'] + lineup['C'] + lineup['G'] +
                                                                     lineup['F']):
            lineup['Util'].append(player['PLAYER_NAME'])
            positions['Util'] -= 1

        if all(count == 0 for count in positions.values()):
            break

    total_score = sum(merged_df.loc[merged_df['PLAYER_NAME'].isin(lineup[pos]), 'total_fantasy_points'].sum() for pos in lineup.keys())
    total_salary = sum(merged_df.loc[merged_df['PLAYER_NAME'].isin(lineup[pos]), 'salary'].sum() for pos in lineup.keys())

    # print the lineup and the total score and total salary
    print('Lineup:')
    for position, players in lineup.items():
        print(f'{position}: {", ".join(players)}')
    print(f'Total score: {total_score}')
    print(f'Total salary: {total_salary}')

optimize_lineup(merged_df)


Lineup:
PG: Damian Lillard
SG: Duncan Robinson
SF: Matisse Thybulle
PF: Kevin Durant
C: Christian Wood
G: Dejounte Murray
F: Aaron Gordon
Util: Clint Capela
Total score: 275.75
Total salary: 58200.0


## Fixing merged_df[POS] column

In [8]:
merged_df.columns.values

array(['PLAYER_NAME', 'PRED_SCORE', 'salary', 'POS',
       'total_fantasy_points'], dtype=object)

In [9]:
merged_df['POS'].unique()

array(['PF', 'C', 'PG', 'SG', 'SF'], dtype=object)

In [10]:
pos_map = {'PF': 'PF, F, UTIL', 'C': 'C, UTIL', 'PG': 'PG, G, UTIL', 'SG': 'SG, G, UTIL', 'SF': 'SF, F, UTIL'}
merged_df['POS'] = merged_df['POS'].replace(pos_map)


In [11]:
import re

# Define a regular expression pattern to split the 'POS' column on commas
pattern = re.compile(r'\s*,\s*')

# Convert the 'POS' column to a tuple using the regular expression pattern to split the values
merged_df['POS'] = merged_df['POS'].apply(lambda x: tuple(pattern.split(x)))


In [12]:
merged_df

,PLAYER_NAME,PRED_SCORE,salary,POS,total_fantasy_points
0,Aaron Gordon,26.337820,7400.0,"(PF, F, UTIL)",27.50
1,Bam Adebayo,36.686985,8600.0,"(C, UTIL)",35.25
2,Bismack Biyombo,23.806379,4000.0,"(C, UTIL)",27.25
3,Chris Silva,15.596848,3400.0,"(C, UTIL)",7.25
4,Christian Wood,25.886425,8700.0,"(C, UTIL)",46.00
5,Clint Capela,31.027426,6700.0,"(C, UTIL)",43.25
6,Damian Lillard,27.056828,9200.0,"(PG, G, UTIL)",36.25
7,DeAndre Jordan,27.208178,4500.0,"(C, UTIL)",18.75
8,Dejounte Murray,32.388432,7800.0,"(PG, G, UTIL)",35.50
9,Derrick Rose,13.274501,5200.0,"(PG, G, UTIL)",28.75


In [13]:
merged_df.to_csv('merged_df_final.csv')

In [14]:
merged_df_final = pd.read_csv('merged_df_final.csv')

In [15]:
merged_df_final

,Unnamed: 0,PLAYER_NAME,PRED_SCORE,salary,POS,total_fantasy_points
0,0,Aaron Gordon,26.337820,7400.0,"('PF', 'F', 'UTIL')",27.50
1,1,Bam Adebayo,36.686985,8600.0,"('C', 'UTIL')",35.25
2,2,Bismack Biyombo,23.806379,4000.0,"('C', 'UTIL')",27.25
3,3,Chris Silva,15.596848,3400.0,"('C', 'UTIL')",7.25
4,4,Christian Wood,25.886425,8700.0,"('C', 'UTIL')",46.00
5,5,Clint Capela,31.027426,6700.0,"('C', 'UTIL')",43.25
6,6,Damian Lillard,27.056828,9200.0,"('PG', 'G', 'UTIL')",36.25
7,7,DeAndre Jordan,27.208178,4500.0,"('C', 'UTIL')",18.75
8,8,Dejounte Murray,32.388432,7800.0,"('PG', 'G', 'UTIL')",35.50
9,9,Derrick Rose,13.274501,5200.0,"('PG', 'G', 'UTIL')",28.75


# Memoization-based solution

In [16]:
def maximize_score(remaining_roles, current_cost, players, memo, lineup):
    if not remaining_roles:
        return 0, lineup

    if current_cost > 50000:
        return float('-inf'), lineup

    if not players:
        return float('-inf'), lineup

    state = (tuple(remaining_roles), current_cost)
    if state in memo:
        return memo[state]

    player = players[0]
    name, cost, predicted_score, valid_roles = player

    remaining_players = players[1:]

    # Calculate score by selecting the current player
    select_player_score, selected_lineup = float('-inf'), lineup
    if current_cost + cost <= 50000:
        for role in set(remaining_roles) & set(valid_roles):
            new_remaining_roles = list(remaining_roles)
            new_remaining_roles.remove(role)
            new_lineup = lineup.copy()
            new_lineup[role] = name
            score, lineup_result = maximize_score(new_remaining_roles, current_cost + cost, 
                                                  remaining_players, memo, new_lineup)
            if predicted_score + score > select_player_score:
                select_player_score = score + predicted_score
                selected_lineup = lineup_result

    # Calculate score by not selecting the current player
    skip_player_score, skip_lineup = maximize_score(remaining_roles, current_cost, remaining_players, 
                                                    memo, lineup)

    # Store the maximum score in the memoization cache
    if select_player_score > skip_player_score:
        memo[state] = (select_player_score, selected_lineup)
    else:
        memo[state] = (skip_player_score, skip_lineup)

    return memo[state]

In [17]:
# Load data from a CSV file
merged_df_2 = merged_df

# Set the 'PLAYER_NAME' column as the index
merged_df_2.set_index('PLAYER_NAME', inplace=True)

# Convert the dataframe to a list of tuples
players = merged_df_2.reset_index()[['PLAYER_NAME', 'salary', 'PRED_SCORE', 'POS']].apply(tuple, axis=1).tolist()

# Define the roles
roles = ["PG", "SG", "SF", "PF", "C", "G", "F", "UTIL"]

# Sort players by predicted score in descending order
sorted_players = sorted(players, key=lambda x: x[2], reverse=True)

# Initialize memoization cache
memo = {}

# Get the maximum predicted fantasy score
# Initialize an empty lineup
initial_lineup = {role: None for role in roles}

# Get the maximum predicted fantasy score and the lineup
max_score, optimal_lineup = maximize_score(roles, 0, sorted_players, memo, initial_lineup)
print("Maximum Predicted Fantasy Score:", max_score)
print("Optimal Lineup:", optimal_lineup)

# Calculate the total cost of the optimal lineup
total_cost = sum(merged_df_2.loc[optimal_lineup[role], 'salary'] for role in roles)
print("Total Cost of Optimal Lineup:", total_cost)


Maximum Predicted Fantasy Score: 221.48938179016113
Optimal Lineup: {'PG': 'Dejounte Murray', 'SG': 'Max Strus', 'SF': 'Yuta Watanabe', 'PF': 'Kevin Durant', 'C': 'Bam Adebayo', 'G': 'Kyle Lowry', 'F': 'Matisse Thybulle', 'UTIL': "Devonte' Graham"}
Total Cost of Optimal Lineup: 49500.0


In [18]:
df_ex = df[(df['GAME_DATE'] == '2021-05-01')]
merged_df = pd.merge(pred, df_ex[['PLAYER_NAME', 'salary', 'POS', 'total_fantasy_points']], on='PLAYER_NAME')
merged_df = merged_df.drop_duplicates(subset=['PLAYER_NAME'], keep='first')
merged_df['POS'].unique()
pos_map = {'PF': 'PF, F, UTIL', 'C': 'C, UTIL', 'PG': 'PG, G, UTIL', 'SG': 'SG, G, UTIL', 'SF': 'SF, F, UTIL'}
merged_df['POS'] = merged_df['POS'].replace(pos_map)
import re
# Define a regular expression pattern to split the 'POS' column on commas
pattern = re.compile(r'\s*,\s*')
# Convert the 'POS' column to a tuple using the regular expression pattern to split the values
merged_df['POS'] = merged_df['POS'].apply(lambda x: tuple(pattern.split(x)))
merged_df

,PLAYER_NAME,PRED_SCORE,salary,POS,total_fantasy_points
0,Alex Len,7.247935,3500.0,"(C, UTIL)",7.25
1,Bam Adebayo,36.686985,8800.0,"(C, UTIL)",40.00
2,Bismack Biyombo,23.806379,3100.0,"(C, UTIL)",30.75
3,Christian Wood,25.886425,8700.0,"(C, UTIL)",22.75
4,Clint Capela,31.027426,8600.0,"(C, UTIL)",30.25
5,Daniel Gafford,10.925137,4200.0,"(C, UTIL)",21.25
6,Darius Bazley,22.740122,6400.0,"(PF, F, UTIL)",16.50
9,Dwight Powell,19.063290,3300.0,"(C, UTIL)",11.25
10,Isaiah Stewart,12.315647,6500.0,"(C, UTIL)",17.00
11,Ivica Zubac,12.288687,5300.0,"(C, UTIL)",15.25


In [19]:
# Load data from a CSV file
merged_df_2 = merged_df

# Set the 'PLAYER_NAME' column as the index
merged_df_2.set_index('PLAYER_NAME', inplace=True)

# Convert the dataframe to a list of tuples
players = merged_df_2.reset_index()[['PLAYER_NAME', 'salary', 'total_fantasy_points', 'POS']].apply(tuple, axis=1).tolist()

# Define the roles
roles = ["PG", "SG", "SF", "PF", "C", "G", "F", "UTIL"]

# Sort players by predicted score in descending order
sorted_players = sorted(players, key=lambda x: x[2], reverse=True)

# Initialize memoization cache
memo = {}

# Get the maximum predicted fantasy score
# Initialize an empty lineup
initial_lineup = {role: None for role in roles}

# Get the maximum predicted fantasy score and the lineup
max_score, optimal_lineup = maximize_score(roles, 0, sorted_players, memo, initial_lineup)
print("Maximum Predicted Fantasy Score:", max_score)
print("Optimal Lineup:", optimal_lineup)

# Calculate the total cost of the optimal lineup
total_cost = sum(merged_df_2.loc[optimal_lineup[role], 'salary'] for role in roles)
print("Total Cost of Optimal Lineup:", total_cost)


Maximum Predicted Fantasy Score: 259.25
Optimal Lineup: {'PG': 'Ricky Rubio', 'SG': 'Frank Jackson', 'SF': 'Tim Hardaway Jr.', 'PF': 'JaMychal Green', 'C': 'Karl-Anthony Towns', 'G': 'Russell Westbrook', 'F': 'Justin Holiday', 'UTIL': 'Khem Birch'}
Total Cost of Optimal Lineup: 47500.0


# Greedy Algorithm

In [20]:
def greedy_maximize_score(remaining_roles, current_cost, players, lineup):
    total_predicted_score = 0

    for role in remaining_roles:
        best_player = None
        best_score = float('-inf')

        for i, player in enumerate(players):
            name, cost, predicted_score, valid_roles = player

            # Check if the current player can be selected for the current role
            if role in valid_roles and current_cost + cost <= 50000:
                if predicted_score > best_score:
                    best_player = i
                    best_score = predicted_score

        if best_player is not None:
            player = players.pop(best_player)
            name, cost, predicted_score, _ = player
            lineup[role] = name
            current_cost += cost
            total_predicted_score += predicted_score

    return total_predicted_score, lineup

# Load data from a CSV file
merged_df_2 = merged_df_final

# Set the 'PLAYER_NAME' column as the index
merged_df_2.set_index('PLAYER_NAME', inplace=True)

# Convert the dataframe to a list of tuples
players = merged_df_2.reset_index()[['PLAYER_NAME', 'salary', 'PRED_SCORE', 'POS']].apply(tuple, axis=1).tolist()

# Define the roles
roles = ["PG", "SG", "SF", "PF", "C", "G", "F", "UTIL"]

# Sort players by predicted score in descending order
sorted_players = sorted(players, key=lambda x: x[2], reverse=True)

# Initialize an empty lineup
initial_lineup = {role: None for role in roles}

# Get the maximum predicted fantasy score and the lineup
max_score, optimal_lineup = greedy_maximize_score(roles, 0, sorted_players, initial_lineup)
print("Maximum Predicted Fantasy Score:", max_score)
print("Optimal Lineup:", optimal_lineup)

# Calculate the total cost of the optimal lineup
total_cost = sum(merged_df_2.loc[optimal_lineup[role], 'salary'] for role in roles)
print("Total Cost of Optimal Lineup:", total_cost)


Maximum Predicted Fantasy Score: 198.85574531555176
Optimal Lineup: {'PG': 'Kyle Lowry', 'SG': 'Max Strus', 'SF': 'Yuta Watanabe', 'PF': 'Kevin Durant', 'C': 'Bam Adebayo', 'G': 'Dejounte Murray', 'F': 'Aaron Gordon', 'UTIL': None}


KeyError: None